In [1]:
import datetime as dt
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
class queueSim:
    def __init__(self, nextTime, servTime, totalCust, startServ, endServ):
        self.nextTime = nextTime
        self.servTime = servTime
        self.totCust = totalCust
        self.startServ = startServ
        self.endServ = endServ
        self.startTime = np.empty([self.totCust], dtype='U5')
        self.endTime = np.empty([self.totCust], dtype='U5')
        self.arriveList = np.empty([self.totCust], dtype='U5')
        
    def simulation(self):
        time = 0

        for i in range(self.totCust):
            self.arriveList[i] = dt.datetime.fromtimestamp(time).strftime('%H:%M')

            time += self.nextTime[i]
            self.startServ[i] = max(time, self.endServ[i])
            self.endServ[i] = self.startServ[i] + self.servTime[i]
        
    def getStartTime(self):
        for i in range(self.totCust):
            temp = self.startServ[i]
            self.startTime[i] = dt.datetime.fromtimestamp(temp).strftime('%H:%M')
        
        return np.concatenate((self.startTime.reshape([self.totCust, 1]), np.array(['Arrive'] * self.totCust).reshape([self.totCust, 1])), axis=1)
    
    def getEndTime(self):
        for i in range(self.totCust):
            temp = self.endServ[i]
            self.endTime[i] = dt.datetime.fromtimestamp(temp).strftime('%H:%M')
        
        return np.concatenate((self.endTime.reshape([self.totCust, 1]), np.array(['Depart'] * self.totCust).reshape([self.totCust, 1])), axis=1)
    
    def getTimeSeq(self):
        timeSeq = np.sort(np.concatenate((self.getStartTime(), self.getEndTime())))
        
        return timeSeq

In [20]:
totalCust = 100
nextArrTime = np.random.uniform(4, 10, totalCust).astype(int) * 60
serviceTime = np.random.uniform(3, 9, totalCust).astype(int) * 60
startService = np.zeros([totalCust]).astype(int)
endService = np.zeros([totalCust]).astype(int)

In [21]:
queue1 = queueSim(nextArrTime, serviceTime, totalCust, startService, endService)
queue1.simulation()

In [22]:
df = pd.DataFrame(queue1.getTimeSeq(), columns=['Time', 'Activity']).sort_values(by=['Time'], ignore_index=True)

In [23]:
custInTime = np.zeros([totalCust * 2]).astype(int)
serverStatus = np.empty([totalCust * 2], dtype='U9')
temp = 0

for i in range(totalCust * 2):
    if df.loc[i, 'Activity'] == 'Arrive':
        temp += 1
    
    else:
        temp -= 1
        
    custInTime[i] = temp
    
    if custInTime[i] == 0:
        serverStatus[i] = 'AVAILABLE'
    
    else:
        serverStatus[i] = 'BUSY'

In [24]:
df['numberOfCustomers'] = custInTime
df['serverStatus'] = serverStatus

In [25]:
df

,Time,Activity,numberOfCustomers,serverStatus
0,07:05,Arrive,1,BUSY
1,07:10,Arrive,2,BUSY
2,07:12,Depart,1,BUSY
3,07:17,Arrive,2,BUSY
4,07:18,Depart,1,BUSY
...,...,...,...,...
195,16:58,Depart,0,AVAILABLE
196,16:59,Arrive,1,BUSY
197,17:04,Depart,0,AVAILABLE
198,17:07,Arrive,1,BUSY
